In [1]:
# IMPORTAÇÕES

from consumidor import consumer, atualizacao_carteira
from relatorio import analise_tecnica, grafico, noti_telegram
import yfinance as yf
import finplot as fplt
import numpy as np
import pandas as pd
import mysql.connector
import datetime
import pytz
import os 
from dotenv import load_dotenv, find_dotenv


# CONEXÃO COM BANCO DE DADOS
load_dotenv(find_dotenv())

conn = mysql.connector.connect(
  host= os.getenv('host'),
  user= os.getenv('user'),
  password= os.getenv('password'),
  database="bolsa_valores"
)
cursor = conn.cursor()

# ATUALIZAÇÃO DA BASE DE DADOS DE COTAÇÕES

#consumer()

base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)', conn)





# FORMAÇÃO DA CARTEIRA FUNDAMENTALISTA
dt = datetime.date.today()
try:
    if datetime.date(dt.year,dt.month,dt.day) == datetime.date(dt.year,6,1) or (not os.path.exists('carteira.csv')):
        ativos = base['SYMBOL'].values.tolist()
        list_melhores = []
        for ativo in ativos:
            if atualizacao_carteira(ativo=ativo, base=base):
                list_melhores.append(ativo)
        pd.Series(list_melhores).to_csv('carteira.csv', index=False)
except:
    pass


#PARAMETROS PARA BOT TELEGRAM
token = os.getenv('token')
chat_id = os.getenv('chat_id')

# ANALISE TÉCNICA E NOTIFICAÇÃO 
carteira = list(pd.read_csv('carteira.csv',delimiter=";"))
for ativo in carteira:
    analise = base.loc[base['SYMBOL'] == ativo]
    if analise_tecnica(analise):
        grafico(analise)
        noti_telegram(token= token, chat_id = chat_id, ativo = ativo)











C:\Users\igorb\AppData\Local\Temp\ipykernel_13792\705656545.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)', conn)
c:\Users\igorb\Documents\GitHub\Python\TCC\relatorio\__init__.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chamada_api['m_vol'] = chamada_api.VOLUME.ewm(span=24).mean()
c:\Users\igorb\Documents\GitHub\Python\TCC\relatorio\__init__.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_